In [1]:
import os
import json
import pandas as pd
import traceback

In [9]:
from langchain.chat_models import ChatOpenAI

In [13]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [14]:
KEY=os.getenv("OPENAI_API_KEY")

In [17]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

c:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\GenAi-venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [15]:
KEY

'setx OPENAI_API_KEY your-api-key-here'

In [18]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000014FB5B9F670>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014FB5BC5480>, temperature=0.5, openai_api_key='setx OPENAI_API_KEY your-api-key-here', openai_proxy='')

In [16]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [21]:
with open(r"C:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\Response.json", "r") as f:
    RESPONSE_JSON=json.load(f)

In [22]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [23]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [31]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template=TEMPLATE
)

In [32]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [33]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [34]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
    )

In [35]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [36]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    output_variables=["quiz", "review"], 
    verbose=True,
    )

In [38]:
PATH = r"C:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\data.txt"

In [39]:
print(PATH)

C:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\data.txt


In [42]:
with open(PATH, "r") as file:
    TEXT=file.read()    

In [43]:
TEXT

'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to: \n\n\nSystems, architecture and infrastructure for Gen AI\nFoundation models and their applications\nMulti-modal Gen AI \nRetrieval Augmented Generated\nGen AI based plugins and agents\nLearning from human preferences \nEvaluation of f

In [44]:
##MCQ ko generate krwana hai